In [1]:
%%init_spark
launcher.master = "local[6]"
launcher.driver_memory = '8g'

In [2]:
import org.apache.spark.sql.functions._

Intitializing Scala interpreter ...

Spark Web UI available at http://pc:4040
SparkContext available as 'sc' (version = 3.1.2, master = local[6], app id = local-1631802296514)
SparkSession available as 'spark'


import org.apache.spark.sql.functions._


In [3]:
val rawDataPath = "/mnt/a/Projects/101DataSets/Open-Academic-Graph/raw-data/"
val dataPath = "/mnt/a/Projects/101DataSets/Open-Academic-Graph/data/"

rawDataPath: String = /mnt/a/Projects/101DataSets/Open-Academic-Graph/raw-data/
dataPath: String = /mnt/a/Projects/101DataSets/Open-Academic-Graph/data/


## Read Data

In [4]:
val papers = spark.read.parquet(rawDataPath+"spark/papers.parquet")

papers: org.apache.spark.sql.DataFrame = [authors: array<struct<id:string,name:string>>, doc_type: string ... 11 more fields]


In [6]:
papers.printSchema

root
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- doc_type: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- id: string (nullable = true)
 |-- issue: string (nullable = true)
 |-- n_citation: long (nullable = true)
 |-- page_end: string (nullable = true)
 |-- page_start: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- title: string (nullable = true)
 |-- venue: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- raw: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- year: long (nullable = true)



In [8]:
papers.show(2,false)

+--------------------------------------------------------------------------------------------------------------------------------------------------------+--------+----+----------+-----+----------+--------+----------+--------------------------------------------------------------+-----------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------+------+----+
|authors                                                                                                                                                 |doc_type|doi |id        |issue|n_citation|page_end|page_start|publisher                                                     |title                                                                              |venue                                                                                                           |volume|year|
+---------------------

In [9]:
papers
.groupBy($"doc_type")
.count
.coalesce(1)
.write.option("header","true").mode("overwrite").csv(dataPath+"doc_type_count.csv")

In [10]:
papers.count

res5: Long = 208915369


In [15]:
papers
.groupBy($"year",$"doc_type",$"n_citation")
.count
.coalesce(1)
.write.option("header","true").mode("overwrite")
.csv(dataPath+"year_doc_type_n_citation_count.csv")

In [17]:
papers
.groupBy($"page_start",$"page_end",$"n_citation")
.count
.coalesce(1)
.write.option("header","true").mode("overwrite")
.csv(dataPath+"page_start_page_end_n_citation_count.csv")

In [21]:
papers
.withColumn("author_number",size($"authors") )
.groupBy($"year",$"author_number",$"n_citation",$"doc_type")
.count
.coalesce(1)
.write.option("header","true").mode("overwrite")
.csv(dataPath+"year_doc_type_n_citation_author_number_count.csv")

In [13]:
papers
.select("venue")
.show(false)

+-----------------------------------------------------------------------------------------------------------------------------------------+
|venue                                                                                                                                    |
+-----------------------------------------------------------------------------------------------------------------------------------------+
|{null, Justicia laboral: revista de Derecho del Trabajo y de la Seguridad Social}                                                        |
|{null, Schagen, I. van, Veiligheidscultuur in Nederland : bijdragen aan het seminar van 12 november 2002, 51-53}                         |
|{205083296, IEEE Power & Energy Magazine}                                                                                                |
|{130009153, Pharmaceutical Biology}                                                                                                      |
|{43790123, Advances